#### Enabling Scrolling in the output area.


In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

### Importing Libraries

In [2]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
np.random.seed(24)

## TO Display all the columns and rows in the output area.
pd.options.display.max_columns = None
pd.options.display.max_rows = None

C:\Users\kondavee\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Fetching Data

In [3]:
reference_df = pd.read_excel('C:\\Users\\kondavee\\Desktop\\myRepo\\Personal\\Juniper\\Data-for-Ex1-restaurant-nophone.xlsx', sheetname='reference')
query_df = pd.read_excel('C:\\Users\\kondavee\\Desktop\\myRepo\\Personal\\Juniper\\Data-for-Ex1-restaurant-nophone.xlsx', sheetname='query')

C:\Users\kondavee\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [4]:
query_df.head()

,id,name,address,city,cuisine,reference_id,score(optional)
0,1001,chanterelle,2 harrison st.,new york city,french (new),31.0,70.0
1,1002,daniel,20 e. 76th st.,new york city,french (new),32.0,90.0
2,1003,dawat,210 e. 58th st.,new york city,indian,33.0,90.0
3,1004,felidia,243 e. 58th st.,new york city,italian,NaN,NaN
4,1005,four seasons,99 e. 52nd st.,new york city,american (new),NaN,NaN


In [5]:
reference_df.head()

,id,name,address,city,cuisine
0,1,arnie morton's of chicago,435 s. la cienega blv.,los angeles,american
1,2,art's delicatessen,12224 ventura blvd.,studio city,american
2,3,hotel bel-air,701 stone canyon rd.,bel air,californian
3,4,cafe bizou,14016 ventura blvd.,sherman oaks,french
4,5,campanile,624 s. la brea ave.,los angeles,american


### Query Checker Function

In [6]:
def checker_func(reference_df, query_df):
    """

    :param reference_df: Reference DataFrame
    :param query_df: Quereyed DataFrame.
    :return: resultant DataFrame.
    """
    
    final_df= pd.DataFrame()
    for idx, row in query_df.iterrows():
        name = row['name']
        add = str(row['address']).strip()
        city = str(row['city']).strip()
        cuisine = str(row['cuisine']).strip()

        df_ref = reference_df[reference_df['name'] == name]

        for idx, row_ref in df_ref.iterrows():
            ref_add = str(row_ref['address']).strip()
            ref_city = str(row_ref['city']).strip()
            ref_cuisine = str(row_ref['cuisine']).strip()
            score_add = fuzz.token_set_ratio(add.lower(),ref_add.lower())
            score_city = fuzz.token_set_ratio(city.lower(),ref_city.lower())
            score_cuisine = fuzz.token_set_ratio(cuisine.lower(),ref_cuisine.lower())
            final_score = (score_add + score_city + score_cuisine)/3
            temp = pd.DataFrame({'id': [row['id']] ,'name': [name], 'address': [add], 'city': [city], 'cuisine': [cuisine], 'reference_id': [row_ref['id']],'score': final_score})
            final_df = final_df.append(temp)
    final_df = final_df.reset_index(drop=True)
    return final_df

In [7]:
result_df = checker_func(reference_df, query_df)

### Results

In [8]:
def _color_red_or_green(val):
    color = 'red' if val < 80 else 'green'
    return 'color: %s' % color

final_result_df = result_df.style.applymap(_color_red_or_green, subset=['score'])

In [9]:
final_result_df

,id,name,address,city,cuisine,reference_id,score
0,1001,chanterelle,2 harrison st.,new york city,french (new),31,74
1,1002,daniel,20 e. 76th st.,new york city,french (new),32,100
2,1003,dawat,210 e. 58th st.,new york city,indian,33,85
3,1004,felidia,243 e. 58th st.,new york city,italian,34,100
4,1006,gotham bar & grill,12 e. 12th st.,new york city,american (new),36,100
5,1007,gramercy tavern,42 e. 20th st.,new york city,american (new),37,100
6,1008,island spice,402 w. 44th st.,new york city,caribbean,38,100
7,1009,jo jo,160 e. 64th st.,new york city,french bistro,39,73
8,1010,la caravelle,33 w. 55th st.,new york city,french (classic),40,100
9,1011,la cote basque,60 w. 55th st.,new york city,french (classic),41,100


In [10]:
final_result_df.to_excel("Result_Data_Ex1_Restaurant.xlsx", index=False)